In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

| | |
|-|-|
|Author(s) | [Lavi Nigam](https://github.com/lavinigam-gcp) |

Notebooks:
**goo.gle/io24-gemini-api**

Google AI Cookbook:
**goo.gle/google-ai-cookbook**

Vertex AI Cookbook:
**goo.gle/vertex-ai-cookbook**

# Step 1 - Google AI for Developers Gemini API

In [2]:
# Library installation
! pip install PyPDF2
! apt-get install poppler-utils
! pip install pdf2image
! pip install --upgrade google-generativeai

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



In [ ]:
# Downloading data from Google Drive
!gdown --folder https://drive.google.com/drive/folders/1RfMScB0Y1LUQdW5tvjyYA4_D21H5HMaT?usp=sharing -O /content/data/
print("Download completed")

Download completed


In [1]:
# Library
from IPython.display import display, Markdown, Latex
import google.generativeai as genai
import PyPDF2
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
import textwrap

In [2]:
# Authentication
from google.colab import userdata
genai.configure(api_key=userdata.get('api_key'))

**[Gemini 1.5 Flash](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash)**


![](https://storage.googleapis.com/github-repo/io24/gemini-api-session/img/model_1.png)

**RPM:** Requests per minute

**TPM:** Tokens per minute

**RPD:** Requests per day

**TPD:** Tokens per day

**[Gemini 1.5 Pro](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-pro)**

![](https://storage.googleapis.com/github-repo/io24/gemini-api-session/img/model_2.png)

Gemini models are available in either preview or stable versions. In your code, you can use one of the following model name formats to specify which model and version you want to use.

* **Latest:** Points to the cutting-edge version of the model for a specified generation and variation. The underlying model is updated regularly and might be a preview version. Only exploratory testing apps and prototypes should use this alias.

    To specify the latest version, use the following pattern: `<model>-<generation>-<variation>-latest`. For example, `gemini-1.5-pro-latest`.

* **Latest stable:** Points to the most recent stable version released for the specified model generation and variation.

    To specify the latest stable version, use the following pattern: `<model>-<generation>-<variation>`. For example, `gemini-1.5-pro`.

* **Stable:** Points to a specific stable model. Stable models don't change. Most production apps should use a specific stable model.

    To specify a stable version, use the following pattern: `<model>-<generation>-<variation>-<version>`. For example, `gemini-1.5-pro-001`.

[More detail](https://ai.google.dev/gemini-api/docs/models/gemini#model-versions)


Gemini models are trained to work with the following languages: [Link](https://ai.google.dev/gemini-api/docs/models/gemini#available-languages)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Here is the list of currently available models: [Link](https://ai.google.dev/gemini-api/docs/models/gemini#model-variations)

<div class="alert alert-block alert-warning">
<b>⚠️ A token is equivalent to about 4 characters for Gemini models. 100 tokens are about 60-80 English words. ⚠️</b>
</div>


In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### API

#### Basic Usage

In [ ]:
# Set up the model
# Learn more about the API: https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python
# More details about the models: https://ai.google.dev/gemini-api/docs/models/gemini

from google.generativeai.types import HarmCategory, HarmBlockThreshold, GenerationConfig

generation_config = GenerationConfig(
    temperature=1,
    max_output_tokens=8192,
)

# Explore more about safety settings: https://ai.google.dev/gemini-api/docs/safety-settings
# Explore Gemini's Safety guidance: https://ai.google.dev/gemini-api/docs/safety-guidance
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

# works with text, code, images, video(with or without audio) and audio(mp3) with 2M input context - complex reasoning
model = genai.GenerativeModel(model_name="gemini-1.5-pro-001",  # using stable (not latest)
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - faster inference
model_flash = genai.GenerativeModel(model_name="gemini-1.5-flash-001", # using stable (not latest)
                              generation_config=generation_config,
                              safety_settings=safety_settings)

explore more examples of what's possible with Gemini: [Link](https://ai.google.dev/examples)

In [ ]:
%%time

response = model.generate_content("What is the meaning of life?")

CPU times: user 150 ms, sys: 18.5 ms, total: 168 ms
Wall time: 11.4 s


In [ ]:
to_markdown(response.text)

> The meaning of life is a question that has been pondered by philosophers and theologians for centuries. There is no one answer that will satisfy everyone, as the meaning of life is a deeply personal and individual question. 
> 
> Here are some perspectives to consider:
> 
> * **Intrinsic meaning:** Some believe that life itself is inherently meaningful, regardless of external factors. This meaning might come from the experience of existence, the pursuit of knowledge, or the simple act of living.
> * **Extrinsic meaning:** Others find meaning in something outside themselves, such as religion, relationships, work, or a cause they believe in. These external factors provide a sense of purpose and direction.
> * **Creative meaning:** Some people find meaning in creating things, whether it be art, music, writing, or even building a business. The act of creation can be deeply fulfilling and provide a sense of accomplishment.
> * **Nihilism:** This perspective argues that life is inherently meaningless and that there is no objective purpose to our existence. While seemingly bleak, it can also be liberating, allowing individuals to create their own subjective meaning.
> * **Absurdism:**  This view acknowledges the human desire for meaning and the universe's indifference to it. It encourages embracing this absurdity and finding joy in the present moment.
> 
> Ultimately, the meaning of life is what you make it. It's a journey of exploration and discovery, and the answers you find will likely evolve over time.  
> 
> Instead of searching for a definitive answer, you might consider asking yourself:
> 
> * What brings me joy and fulfillment? 
> * What impact do I want to have on the world?
> * What values do I want to live by? 
> 
> By exploring these questions, you can begin to define your own personal meaning of life. 


In [ ]:
%%time

response = model_flash.generate_content("What is the meaning of life?")

CPU times: user 89.3 ms, sys: 12.9 ms, total: 102 ms
Wall time: 6.43 s


In [ ]:
to_markdown(response.text)

> The meaning of life is a question that has been pondered by philosophers and individuals for centuries. There is no single, universally accepted answer, as the meaning is deeply personal and subjective. 
> 
> However, here are some common perspectives:
> 
> **Existentialism:** Emphasizes individual freedom and responsibility. It suggests that life has no inherent meaning, and individuals must create their own meaning through their choices and actions.
> 
> **Nihilism:**  Believes that life is meaningless and purposeless. It emphasizes the absurdity and futility of existence.
> 
> **Absurdism:**  Accepts the absurdity of life but encourages individuals to embrace it and find meaning in the face of meaninglessness.
> 
> **Religious and Spiritual Beliefs:** Many religions and spiritual practices offer a framework for understanding the meaning of life. They often focus on concepts like serving a higher power, achieving enlightenment, or fulfilling a divine purpose.
> 
> **Humanism:**  Focuses on human values, reason, and ethics. It suggests that meaning can be found in human relationships, creativity, and contributions to society.
> 
> **Hedonism:**  Emphasizes the pursuit of pleasure and happiness as the ultimate goal of life.
> 
> **Stoicism:**  Encourages individuals to accept what they cannot control and focus on what they can. It emphasizes virtue, reason, and living in accordance with nature.
> 
> **Other Perspectives:** 
> * **Finding happiness and fulfillment:** Many people find meaning in pursuing happiness, personal growth, and fulfillment in their lives.
> * **Leaving a positive impact:** Some believe that finding meaning involves contributing to the world in a positive way, through acts of kindness, service, or creativity.
> * **Experiencing life:** Others find meaning in simply experiencing life to the fullest, embracing its joys and sorrows, and appreciating its complexities.
> 
> Ultimately, the meaning of life is a personal journey of discovery. It's a question that each individual must answer for themselves. There is no right or wrong answer, and the meaning of life can evolve and change over time. 


In [ ]:
response.candidates

[content {
  parts {
    text: "The meaning of life is a question that has been pondered by philosophers and individuals for centuries. There is no single, universally accepted answer, as the meaning is deeply personal and subjective. \n\nHowever, here are some common perspectives:\n\n**Existentialism:** Emphasizes individual freedom and responsibility. It suggests that life has no inherent meaning, and individuals must create their own meaning through their choices and actions.\n\n**Nihilism:**  Believes that life is meaningless and purposeless. It emphasizes the absurdity and futility of existence.\n\n**Absurdism:**  Accepts the absurdity of life but encourages individuals to embrace it and find meaning in the face of meaninglessness.\n\n**Religious and Spiritual Beliefs:** Many religions and spiritual practices offer a framework for understanding the meaning of life. They often focus on concepts like serving a higher power, achieving enlightenment, or fulfilling a divine purpose.\n\

Explore API Troubleshooting and API errors here: [Link](https://ai.google.dev/gemini-api/docs/troubleshooting)

#### Streaming

In [ ]:
%%time
response = model_flash.generate_content("What is the meaning of life?", stream=True)

CPU times: user 86.9 ms, sys: 2.54 ms, total: 89.5 ms
Wall time: 5.78 s


In [ ]:
for chunk in response:
  print(chunk.text)
  print("_"*80)

The
________________________________________________________________________________
 meaning of life is a question that has been pondered by philosophers and theologians for centuries
________________________________________________________________________________
, and there is no single, universally accepted answer.  It's a
________________________________________________________________________________
 highly personal question, and the meaning you find may be different from what others find.

Here are some perspectives:

* **Existentialism:** This philosophical
________________________________________________________________________________
 movement emphasizes individual freedom and responsibility. The meaning of life is not predetermined but created by our choices and actions. We are free to define our own purpose and values
________________________________________________________________________________
.
* **Nihilism:** This philosophy suggests that life is inherently meaningl

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The meaning of life is a question that has been pondered by philosophers and theologians for centuries, and there is no single, universally accepted answer.  It's a highly personal question, and the meaning you find may be different from what others find.\n\nHere are some perspectives:\n\n* **Existentialism:** This philosophical movement emphasizes individual freedom and responsibility. The meaning of life is not predetermined but created by our choices and actions. We are free to define our own purpose and values.\n* **Nihilism:** This philosophy suggests that life is inherently meaningless and without purpose. There is no inherent value or objective truth.\n* **Religion and Spirituality:** Many religions and spiritual traditions offer a framework for under

#### Token Count

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
prompt = "What is life?"
response = model.generate_content(prompt)


print("count before response generation: ", model.count_tokens(prompt))
print("\nResponse: ",response.text)
print("\n Usage Metadata:\n",response.usage_metadata)

count before response generation:  total_tokens: 4


Response:  That's a question philosophers have pondered for centuries! There's no single, universally accepted answer, but here are some perspectives:

**Scientific perspective:**

* **Biological definition:**  Life is characterized by a set of shared properties: organization, metabolism, growth, adaptation, response to stimuli, reproduction, and homeostasis.
* **Physical definition:** Life is a complex system of chemical reactions that maintains itself and can reproduce.
* **Information processing:** Some argue that life is fundamentally about information processing and replication.

**Philosophical perspective:**

* **Existentialism:**  Life is what you make it. It's about creating meaning and purpose in a world that is ultimately meaningless.
* **Spiritualism:**  Life is a journey of the soul, connected to a higher power or force.
* **Materialism:**  Life is a product of matter and energy, with no inherent meaning or purpose.

**P

## Testing Hypothesis

### PDF (Extracted Text)

In [ ]:
#@title Helper Functions
## Helper Function

def pdf_to_dict_str(file_path):
    """Reads a PDF file and returns a dictionary with page numbers as keys and page text as values."""

    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        page_dict = {}
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            page_dict[page_num + 1] = page_text  # Page numbers start from 1

        return str(page_dict)

Source File: [Link](https://drive.google.com/file/d/1GXGfwoe8-vsoOM1E8SBOCybc7R5XRd2P/view?usp=drive_link) ~18 pages

In [ ]:
# read through each page and extracts the text and saves in the dictionary as {1:"context of page 1"}, 2:"context of page 2"}, .... }
pdf_context = pdf_to_dict_str('/content/data/Idea/stage_1_prototype_Google Cloud TPU blog.pdf')

In [ ]:
pdf_context[:100] #The whole dict is converted to strings so as to pass as context in the model

"{1: 'Google\\nCloud\\ndemonstrates\\nthe\\nworld’ s\\nlargest\\ndistributed\\ntraining\\njob\\nfor\\nlarge\\nla"

In [ ]:
rich_Markdown(eval(pdf_context)[1]) # Page 1 #use eval to convert string to dict object

Google Cloud demonstrates the world’ s largest distributed training job for large language models across 50000+ TPU
v5e chips November 9, 2023 Rajesh Anantharaman Product Management Lead, Cloud TPU With the boom in generative AI,  
the size of foundational large language models (LLMs) has grown exponentially , utilizing hundreds of billions of  
parameters and trillions of training tokens.

In [ ]:
%%time

prompt_parts = [
  pdf_context,
  "What are key achievement for Google Cloud from the following blog. Mention in bullet.",
]

print(model.count_tokens(prompt_parts))
response = model.generate_content(prompt_parts)
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 7523


 Usage Metadata:
 prompt_token_count: 7524
candidates_token_count: 365
total_token_count: 7889

CPU times: user 414 ms, sys: 16.6 ms, total: 431 ms
Wall time: 7.06 s


Here are the key achievements highlighted in the blog, focusing on Google Cloud's accomplishments:                 

 • World's Largest Distributed LLM Training Job: Google Cloud achieved the largest publicly disclosed LLM training 
   job, utilizing over 50,000 TPU v5e chips. This demonstrates their leadership in providing infrastructure and    
   software for massive AI workloads.                                                                              
 • Cloud TPU Multislice Training:  Google Cloud introduced Multislice Training, a full-stack training offering     
   designed for scalability, reliability, and ease of use. It addresses key challenges in orchestrating, compiling,
   and optimizing large-scale distributed training.                                                                
 • Scalable and Robust Orchestration: Multislice Training enables users to scale LLM training across tens of       
   thousands of TPU chips reliably, with fault tolerance throughout the workflow.                                  
 • Performant Compilation: Multislice Training leverages the XLA compiler for efficient management of compute and  
   communication, maximizing performance and efficiency.                                                           
 • Flexible Stack for End-to-End Training: Multislice Training provides support for popular ML frameworks like JAX 
   and PyTorch, simplifying the training process for developers.                                                   
 • Open-Sourcing Key Components:  Google Cloud open-sourced several critical components of their Multislice        
   Training stack, including XPK, MaxText, AQT, and XLA, fostering collaboration and advancement within the broader
   AI/ML community.                                                                                                
 • Impressive Scaling Efficiency: Google Cloud demonstrated excellent scaling efficiency with their LLM training   
   job, achieving impressive Model FLOPs Utilization (MFU) and Effective Model FLOPs Utilization (EMFU) metrics.   
 • Continuous Improvement: Google Cloud emphasizes ongoing efforts to further improve their software stack and push
   the boundaries of LLM training at scale. They are working on optimizing scheduling, compilation, and exploring  
   scalable solutions for even greater performance.

In [ ]:
%%time

print(model_flash.count_tokens(prompt_parts))
response = model_flash.generate_content(prompt_parts)
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 7523


 Usage Metadata:
 prompt_token_count: 7524
candidates_token_count: 380
total_token_count: 7904

CPU times: user 119 ms, sys: 15 ms, total: 134 ms
Wall time: 7.67 s


Here are some of the key achievements for Google Cloud, as presented in the blog:                                  

 • World's Largest Distributed LLM Training Job: Google Cloud TPU ran the world's largest publicly disclosed       
   distributed training job for LLMs, utilizing over 50,000 TPU v5e chips, achieving a peak performance of 10      
   exa-FLOPs (16-bit) or 20 exa-OPs (8-bit).                                                                       
 • Introduction of Cloud TPU Multislice Training:  A full-stack training offering supporting TPU v4 and v5e,       
   designed for scalability, reliability, and ease of use for end-to-end ML training optimization.                 
 • Open-Sourcing Key Components:  Google open-sourced several key components of the Multislice Training stack,     
   including XPK (Accelerated Processing Kit), MaxText (JAX LLM implementation), AQT (Accurate Quantized Training),
   and XLA (Accelerated Linear Algebra) to contribute to the broader AI/ML community.                              
 • Performance and Efficiency Improvements: Google achieved impressive scaling efficiency with the 50,000+ TPU v5e 
   chips, reaching 66.86% MFU (Model FLOPs Utilization) with BF16 training on a single pod. They also achieved an  
   impressive 5.32 exa-OP/s with INT8 quantized training, demonstrating the benefits of AQT.                       
 • Continuous Development and Innovation: Google is actively working on further improving the software stack,      
   including optimization for job start-up time, scaling efficiency, and compiler advancements.                    
 • Strong Technical Capabilities: The blog highlights Google's expertise in areas such as orchestration,           
   compilation, end-to-end optimization, distributed data loading, and checkpointing, showcasing their leading     
   position in distributed ML training.

Understand your billing: [Link](https://ai.google.dev/gemini-api/docs/billing)

Source File: [Link](https://drive.google.com/file/d/1iT0qe9EU8RQyiAlrMlJ-SH-E_WtMGFNm/view?usp=drive_link) ~15 pages

In [ ]:
%%time
prompt_parts = [
  pdf_to_dict_str('/content/data/Idea/goog-10-q-q2-2023-4-1-15.pdf'),
  """Answer based on the document provided:
- How has Google Cloud performed in last quarters?
- What factors have influenced its numebrs and is it positive or negative?
- What is the leadership view on Google Cloud business?
- What are some changes and decisions they have made along the way to achieve their goals?
- How are they doing with respect to other cloud companies?
  """,
]
print(model_flash.count_tokens(prompt_parts))
response = model.generate_content(prompt_parts)
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 11470


 Usage Metadata:
 prompt_token_count: 11471
candidates_token_count: 541
total_token_count: 12012

CPU times: user 933 ms, sys: 17.9 ms, total: 950 ms
Wall time: 9.92 s


The provided document is a 10-Q filing, which is a quarterly report that publicly traded companies file with the   
Securities and Exchange Commission (SEC). It outlines the company's financial performance but doesn't go into deep 
detail about specific business units like Google Cloud.                                                            

Here's what we can gather from the document about Google Cloud:                                                    

 • Google Cloud Revenue:                                                                                           
    • The document provides revenue figures for Google Cloud:                                                      
       • Three months ended June 30, 2022: $6,276 million                                                          
       • Three months ended June 30, 2023: $8,031 million                                                          
       • Six months ended June 30, 2022: $12,097 million                                                           
       • Six months ended June 30, 2023: $15,485 million                                                           
    • Performance: This shows Google Cloud experienced significant revenue growth in both the three-month and      
      six-month periods.                                                                                           
 • Factors Influencing Google Cloud Revenue:                                                                       
    • The document mentions that "Deferred revenues primarily relate to Google Cloud and Google other". This       
      suggests that Google Cloud revenue is largely driven by long-term contracts with customers who pay in advance
      for services.                                                                                                
    • The document also states that the amount and timing of revenue recognition for these commitments is "largely 
      driven by our ability to deliver in accordance with relevant contract terms and when our customers utilize   
      services." This implies that Google Cloud's performance is influenced by factors like its ability to deliver 
      on these contracts and customer usage patterns.                                                              
 • Leadership View on Google Cloud:                                                                                
    • The document does not provide specific leadership statements about Google Cloud.                             
 • Changes and Decisions:                                                                                          
    • The document doesn't mention specific changes or decisions made to achieve Google Cloud goals.               
 • Comparison with Other Cloud Companies:                                                                          
    • The document doesn't compare Google Cloud to other cloud companies.                                          

To find more specific information on these topics, you'd need to:                                                  

 • Refer to Alphabet's other SEC filings:  The annual report (10-K) often provides more in-depth discussions about 
   the Google Cloud business.                                                                                      
 • Consult industry reports and news articles:  There are many research firms and news outlets that cover the cloud
   computing market and provide analyses of Google Cloud's performance relative to its competitors.                
 • Review Alphabet's earnings call transcripts: These transcripts often contain detailed information and commentary
   from Alphabet's leadership about Google Cloud's performance, strategies, and outlook.

### Audio File

Source File: [Link](https://drive.google.com/file/d/1kaq58S4oRmuZgDqK109uc6mwedVmPAby/view?usp=drive_link) ~1 hour

In [ ]:
%%time

earning_calls = genai.upload_file(path="/content/data/Idea/stage_1_prototype_Alphabet 2023 Q4 Earnings Call.mp3",
                            display_name="earning_calls")

CPU times: user 464 ms, sys: 186 ms, total: 650 ms
Wall time: 4.69 s


In [ ]:
%%time

prompt_parts = [
  earning_calls,
  "what are major comments by Sundar and Ruth about AI and what key questions were asked by analyst and give their name? give answer in bullets ",
]
print(model.count_tokens(prompt_parts))
response = model.generate_content(prompt_parts,request_options={"timeout": 600})
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 109213


 Usage Metadata:
 prompt_token_count: 109214
candidates_token_count: 1214
total_token_count: 110428

CPU times: user 728 ms, sys: 76.9 ms, total: 805 ms
Wall time: 57 s


Sure, here are the major comments about AI from Sundar Pichai and Ruth Porat, as well as key questions about AI    
asked by analysts, with their names.                                                                               

Sundar Pichai (CEO)                                                                                                

 • AI Investments: Alphabet has been a leader in AI, using it to enhance products like Search, Ads, and consumer   
   and enterprise tools.                                                                                           
 • Gemini Era: The launch of Gemini models represents a significant step in AI advancement. These models are       
   designed to understand and integrate text, images, audio, video, and code. Gemini is multimodal and scalable,   
   running on devices from mobile to data centers.                                                                 
 • Gemini in Search:  Gemini is being incorporated into Search Generative Experience (SGE), enhancing speed and    
   enabling the handling of more complex queries involving comparisons and longer questions. It's particularly     
   useful in areas like education.                                                                                 
 • SGE Expansion: SGE is expanding with increased links, wider sources on the results page, and a focus on user    
   value and publisher traffic.                                                                                    
 • AI for Advertisers: Alphabet sees growing interest in its AI-powered advertising solutions, such as a           
   conversational experience for faster campaign creation.                                                         
 • Bard advancements: The conversational AI tool, Bard, is now powered by Gemini Pro, enabling it to excel in tasks
   like understanding, summarizing, reasoning, coding, and planning. Bard is available in over 40 languages and 230
   countries. An advanced version powered by Gemini Ultra is in development.                                       
 • Google One with AI:  AI features are being added to the Google One subscription service to further drive growth.
 • Google Cloud AI: Throughout 2023, thousands of AI product advancements were introduced, including broad         
   generative AI capabilities across infrastructure, the Vertex AI platform, and Duet AI agents.                   
 • AI Infrastructure Investments: Alphabet is investing heavily in state-of-the-art computer infrastructure to     
   support the growth of AI-powered services for both Google and its customers.                                    

Ruth Porat (CFO)                                                                                                   

 • AI-Driven Cloud Growth:  Google Cloud's momentum is fueled by the increasing contribution from AI, including    
   Google Workspace's strong revenue growth.                                                                       
 • AI and Profitability:  The profitability improvements in Google Cloud reflect a sustained focus on leveraging AI
   technology while maintaining healthy profit margins.                                                            
 • AI for Organizational Efficiency: Alphabet is committed to leveraging AI to streamline operations across the    
   company, further optimizing its cost base.                                                                      

Key Analyst Questions About AI                                                                                     

 • Brian Nowak (Morgan Stanley):                                                                                   
    • What are the hurdles and gating factors that will dictate the pace of the broad rollout of generative AI     
      advertising tools? How should we think about when these tools could have a bigger impact on the overall      
      business?                                   

In [ ]:
%%time

print(model_flash.count_tokens(prompt_parts))
response = model_flash.generate_content(prompt_parts)
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 109213


 Usage Metadata:
 prompt_token_count: 109214
candidates_token_count: 703
total_token_count: 109917

CPU times: user 378 ms, sys: 41.5 ms, total: 419 ms
Wall time: 29.1 s


Here are the major comments by Sundar and Ruth about AI, and the key questions asked by analysts, along with their 
names:                                                                                                             

Sundar's Comments on AI                                                                                            

 • Investments in AI: Sundar highlighted Alphabet's strong momentum and product innovation driven by AI,           
   particularly in Search and across various consumer and enterprise products.                                     
 • Gemini: Sundar emphasized Gemini as a significant AI breakthrough, a multimodal model that can understand and   
   combine text, images, audio, video, and code. He mentioned Gemini Ultra is coming soon.                         
 • AI in Search: Sundar discussed how Gemini is being integrated into Search, leading to a faster and more         
   comprehensive search experience, especially for complex queries.                                                
 • AI in Other Products: Sundar highlighted AI-powered solutions for advertisers, including a conversational       
   experience for creating search campaigns and Bard, a conversational AI tool powered by Gemini Pro.              

Ruth's Comments on AI                                                                                              

 • Impact on Google Cloud: Ruth mentioned the increasing contribution of AI to Google Cloud Platform's momentum,   
   with strong adoption of Vertex AI and the introduction of Duet AI agents.                                       
 • Long-Term Growth: Ruth emphasized the importance of AI in driving long-term growth across Alphabet, highlighting
   the significant investments in technical infrastructure and the use of AI to improve efficiency and operations. 

Key Questions Asked by Analysts:                                                                                   

 • Brian Nowak (Morgan Stanley):                                                                                   
    • AI in Advertising: Brian inquired about the hurdles and gating factors that could impact the rollout of new  
      AI advertising tools and their broader impact on the business.                                               
    • Sales Force Intensity: Brian asked about the long-term impact of AI-based tools like PMax on the advertising 
      business and the sales force intensity.                                                                      
 • Doug Anmuth (J.P. Morgan):                                                                                      
    • Cost Structure Re-Engineering: Doug asked about Ruth's assessment of progress made in re-engineering the cost
      structure and any future guidance.                                                                           
    • NFL Sunday Ticket: Doug asked about key learnings from the first year of NFL Sunday Ticket and the expected  
      return on investment.                                                                                        
 • Eric Sheridan (Goldman Sachs):                                                                                  
    • Search Generative Experience Evolution: Eric inquired about the future evolution of traditional search and   
      Google Assistant in a world with search generative experiences like Bard.                                    
    • One-Timers in Q4: Eric sought clarification on the allocation of one-timers in the Q4 results, particularly  
      related to real estate charges.                                                                              
 • Michael Nathanson (MoffettNathanson):                                                                           
    • Connected TV Outside the U.S.: Michael asked about the opportunity for Connected TV outside the U.S. and     
      Alphabet's plans for driving adoption.      

#### Context Caching [Paid Feature]

More details about the API: [Link](https://ai.google.dev/gemini-api/docs/caching?lang=python)

Context Caching pricing: [Link](https://ai.google.dev/pricing)

In [ ]:
%%time

from google.generativeai import caching
import datetime
import time

# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-flash-001',
    display_name='alphabet earning call', # used to identify the cache
    system_instruction=(
        'You are an expert audio analyzer, and your job is to answer '
        'the user\'s query based on the audio file you have access to.'
    ),
    contents=[earning_calls],
    # caching duration is called the time to live (TTL). If not set, the TTL defaults to 1 hour.
    # The cost for caching depends on the input token size and how long you want the tokens to persist.
    ttl=datetime.timedelta(minutes=5),
)

CPU times: user 616 ms, sys: 53.1 ms, total: 669 ms
Wall time: 49.9 s


In [ ]:
%%time

# Construct a GenerativeModel which uses the created cache.
cached_model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = cached_model.generate_content([("""Instructions:
- Quote the speaker name.
- Mention the timestamp.
- Respond in structured format and bullets.

Task: Summarize the questions by Bank of America analyst and their response from Alphabet.
""")])

print(response.usage_metadata)


rich_Markdown(response.text)

prompt_token_count: 109259
candidates_token_count: 481
total_token_count: 109740
cached_content_token_count: 109213

CPU times: user 265 ms, sys: 28.4 ms, total: 294 ms
Wall time: 20.1 s


Here's a summary of the questions asked by the Bank of America analyst and the responses from Alphabet:            

 • Speaker:  Justin Post, Bank of America                                                                          
 • Timestamp: 47:48 - 48:30                                                                                        
 • Question:  It's been about 2 years since PMax was launched. However, there has been a lineage of product        
   development and rollout of things like smart bidding in your history. I believe it was designed to help smaller 
   advertisers more easily run search advertising. Today you are helping them generate creatives and manage their  
   spend and maximize their ROI across multiple global surfaces. How are you feeling about enabling SMBs who       
   otherwise could not advertise with you before? And what kind of tailwind does that create for your revenue      
   growth over the longer term?                                                                                    
 • Response:                                                                                                       
    • SMBs are a huge focus for Alphabet, mentioned several times before.                                          
    • They're part of the GCS channel.                                                                             
    • SMBs have been under a lot of change in the last few years.                                                  
    • Alphabet's focus has been to invest in solutions that level the playing field.                               
    • Feedback they receive is that SMBs need easy solutions that drive value quickly.                             
    • The AI-powered solutions are making workflow easier.                                                         
    • Alphabet is very satisfied with what they're seeing and will continue to invest in the area.                 
 • Speaker: Justin Post, Bank of America                                                                           
 • Timestamp: 47:50 - 48:29                                                                                        
 • Question:  You've been consistent about the strength of Shorts over the past year. You talked a bit about the   
   monetization challenges, do you still see the headwinds that you saw at the beginning? Anything you can share   
   about the density of the auction or advertising interest on Shorts to help us with whether or not that's still a
   headwind to growth?                                                                                             
 • Response:                                                                                                       
    • Shorts were built to respond to a huge demand for short-form videos from both creators and viewers.          
    • Alphabet is pleased with the growth they have seen.                                                          
    • Shorts monetization continues to progress nicely.                                                            
    • Revenue sharing for Shorts has increased every month.                                                        
    • Alphabet is committed to long-term partnerships and sees Shorts as an opportunity.

In [ ]:
# Query the model
response = cached_model.generate_content([("""Instructions:
- Quote the speaker name.
- Mention the timestamp.
- Respond in structured format and bullets.

Task: Summarize the questions by Morgan Stanley analyst and their response from Alphabet.
""")])

print(response.usage_metadata)


rich_Markdown(response.text)

prompt_token_count: 109257
candidates_token_count: 647
total_token_count: 109904
cached_content_token_count: 109213



Here is a summary of the questions and responses from Morgan Stanley analyst:                                      

 • Speaker: Brian Nowak (Morgan Stanley)                                                                           
 • Timestamp: 32:57-33:43                                                                                          
    • Question 1: "Can you walk us through some of the hurdles and gating factors that we should be thinking       
      through that dictate the pace at which these tools can be rolled out broadly?"                               
    • Question 2: "How do you think about the long-term salesforce intensity of the advertising business as you    
      roll more AI-based tools like PMax impacting the overall ad allocation?"                                     
 • Speaker: Philip Schindler (Alphabet)                                                                            
 • Timestamp: 33:45-36:19                                                                                          
    • Response 1: "AI has been at the core of our advertising products for a very long time, and the recent        
      advances are allowing us to drive more value for advertisers across a large range of different areas like    
      bidding, targeting, creative, as well as our core advertiser and publisher experiences. We're seeing in core 
      search ads for example, the bidding side, the value-based bidding, as a very significant one, on the         
      targeting side, the broad match, on creatives, responsive search ads, automatically created ads. We're very  
      happy with the progress we're seeing in those areas. I think I mentioned in one of the previous calls, that  
      when you asked about hurdles and gating factors, the AI essentials play a huge role in this, trying to get   
      everybody ready to really take full advantage of those tools. We talked about the progress on PMax as well,  
      and that we were very happy with in Q4. So, overall, I think we're on track with what we're expecting here." 
    • Response 2: "On the salesforce intensity side, we've done a few reallocations, or let me call it portfolio   
      adjustments. As you know, we have two large, different teams. One is our LCS, large customer solutions team  
      that's really focused on transformational growth for the largest and most sophisticated customers and we have
      our GCS, global customer solutions team, which is the channel where really every customer starts from some of
      our largest customers now to also the millions of smaller ones in SMBs. The GCS is really, then, scaling     
      growth by dynamically delivering really the right treatment for every customer, and importantly to note, it's
      also our fastest-growing channel and it delivered particularly strong growth in Q4. So, we've done           
      adjustments here to focus more resources on the GCS side, but I also want to be clear when we restructure,   
      there's always an opportunity to be more efficient and smarter in how we service and grow our customers.     
      We're not restructuring because AI is taking away roles, that's important here. But we see significant       
      opportunities here with our AI-powered solutions to actually deliver incredible ROI at scale, and that's why 
      we're doing some of those adjustments. So, I look at salesforce intensity as a stronger focus going forward  
      on the channels I mentioned here."

In [ ]:
# delete the file
genai.delete_file(name=earning_calls.name)

### Images (Read PDF as Images)

In [ ]:
#@title Helper Functions
## Helper Function

import os
from pdf2image import convert_from_path

def pdf_to_images(file_path, output_folder="output_folder"):
    """Converts a PDF file into images for each page, saves them to a folder, and returns a list of image paths."""

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert PDF to images
    images = convert_from_path(file_path, dpi=200, output_folder=output_folder, fmt="jpg")

    # Generate image paths
    image_paths = []
    pages_list = []
    upload_file_name = []
    for i, image in enumerate(images):
        image_name = f"page_{i+1}.jpg"  # Page numbers start from 1
        image_path = os.path.join(output_folder, image_name)
        image.save(image_path, "JPEG")
        image_paths.append(image_path)
        files_to_upload = genai.upload_file(image_path, display_name=image_path)
        pages_list.append(files_to_upload)
        upload_file_name.append([files_to_upload.name, files_to_upload.display_name])

    return image_paths, pages_list, upload_file_name

def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file)
  return request

def delete_files(output_folder):
  for filename in os.listdir(output_folder):
    if filename.endswith(".jpg"):
      file_path = os.path.join(output_folder, filename)
      genai.delete_file(name=file_path)

Source File: [Link](https://drive.google.com/file/d/1GXGfwoe8-vsoOM1E8SBOCybc7R5XRd2P/view?usp=drive_link) ~18 pages

In [ ]:
%%time

file_path = "/content/data/Idea/stage_1_prototype_Google Cloud TPU blog.pdf"
output_folder = "/content/output_images"
image_paths, pages_list, upload_file_name = pdf_to_images(file_path, output_folder)

CPU times: user 1.17 s, sys: 437 ms, total: 1.61 s
Wall time: 50.1 s


In [ ]:
image_paths[1]

'/content/output_images/page_2.jpg'

In [ ]:
pages_list[1]

genai.File({
    'name': 'files/xa3g6r2evm1u',
    'display_name': '/content/output_images/page_2.jpg',
    'mime_type': 'image/jpeg',
    'sha256_hash': 'YjhjM2FlYTQ1YjFjODQzMGVhNTFkZTRhZjc5ZTQ0NWRhMDEwNmQyYzNiNjUyODc3NDhjY2EwZDYwZWVhMDU4MQ==',
    'size_bytes': '181004',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/xa3g6r2evm1u',
    'create_time': '2024-07-16T16:44:25.926108Z',
    'expiration_time': '2024-07-18T16:44:25.844202521Z',
    'update_time': '2024-07-16T16:44:25.926108Z'})

In [ ]:
upload_file_name[1]

['files/xa3g6r2evm1u', '/content/output_images/page_2.jpg']

In [ ]:
%%time

response = model_flash.generate_content(
make_request("what is the emfu for bf16 and 128b parameter model with 1 tpu v5e pod? Cite the table and page number and explain the significance of the results",
             pages_list))
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)


 Usage Metadata:
 prompt_token_count: 4684
candidates_token_count: 174
total_token_count: 4858

CPU times: user 49.6 ms, sys: 8.81 ms, total: 58.4 ms
Wall time: 3.22 s


The EMFU for a 128B parameter model with BF16 training on a single TPU v5e pod is 44.67%. This information can be  
found in the table labeled "MaxText LLM Training Results" on page 14 of the document.                              

The EMFU (Effective Model FLOPs Utilization) represents the percentage of the theoretical peak FLOPS (floating     
point operations per second) that is actually being utilized during training.  An EMFU of 44.67% means that the    
model is using 44.67% of the potential computing power of the TPU v5e.                                             

The significance of this result is that it indicates that the training process is not as efficient as it could be. 
There is room for improvement in the utilization of the TPU v5e's capabilities.

In [ ]:
for filename in upload_file_name:
  print("Deleteing...",filename[0])
  genai.delete_file(name=filename[0])

Deleteing... files/mzzdp5t7n8iz
Deleteing... files/xa3g6r2evm1u
Deleteing... files/zq783pjkz2yp
Deleteing... files/rarnd9con98v
Deleteing... files/nofplwcbd0a1
Deleteing... files/ur5so9dmunx
Deleteing... files/ngfn4ylxwjoh
Deleteing... files/hozz3jq0clx8
Deleteing... files/3t8xemrlh836
Deleteing... files/qmzzjg53ye5a
Deleteing... files/cp6hhu3mqnnf
Deleteing... files/yt13h3ds9nib
Deleteing... files/9hql8rovko8g
Deleteing... files/n33zomhutfru
Deleteing... files/uhkcjseh0axl
Deleteing... files/l0mc20evzjct
Deleteing... files/dklxcjnqldkc
Deleteing... files/jvll0wk7dpnw


### Video

Source File: [Link](https://drive.google.com/file/d/19aqpEZqTVTERbySYVrtDgSjuUBUabeSE/view?usp=drive_link)

In [ ]:
%%time
# Upload a video ~ 1min

video_file = genai.upload_file(path="/content/data/Idea/What's next for AI and Google Search _ Google I_O 2023.mp4",
                            display_name="product_pitch_google_video_file")

CPU times: user 52.6 ms, sys: 17.5 ms, total: 70.1 ms
Wall time: 3.36 s


In [ ]:
%%time

prompt_parts = [
  video_file,
  "Describe this video and How is google using generative ai in search? Give response in bullet",
]
print(model.count_tokens(prompt_parts))
response = model.generate_content(prompt_parts,request_options={"timeout": 600})
print("\n Usage Metadata:\n",response.usage_metadata)
rich_Markdown(response.text)

total_tokens: 27157


 Usage Metadata:
 prompt_token_count: 27158
candidates_token_count: 418
total_token_count: 27576

CPU times: user 229 ms, sys: 23.5 ms, total: 252 ms
Wall time: 16.8 s


Sure. The video is about a new feature from Google called "Generative AI in Search," which aims to change how      
people use Google search.                                                                                          

Here's a breakdown of the key points from the video and how Google is using generative AI:                         

 • Generative AI is Changing Search: The video emphasizes that AI has been powering the evolution of search and    
   that it's now moving front and center. This signifies a more prominent and interactive role of AI in delivering 
   search results.                                                                                                 
 • Ask Complex Questions in a Single Search: The video shows how generative AI allows users to ask multi-faceted   
   questions within a single search bar, eliminating the need for multiple searches. For example, a user searches  
   for "dress for an outdoor wedding in Miami in May in a trending color with 2-day delivery."                     
 • AI Provides Contextual and Comprehensive Answers: Instead of just providing links, generative AI analyzes the   
   search query and generates detailed responses that consider various factors. In the dress example, it           
   acknowledges the hot and humid weather in Miami during May and suggests breathable materials and bold colors.   
 • Dynamic Content Generation:  The AI generates a selection of dresses based on the user's criteria, even         
   suggesting options with 2-day delivery.  It also shows suggested follow-up questions based on the search like   
   "matching accessories" and "formal wedding guest dresses."                                                      
 • Integration of Diverse Formats:  The video highlights how generative AI goes beyond text-based responses by     
   incorporating images, videos, and even social media content. For instance, when the user asks about shoes, the  
   AI showcases a relevant TikTok video.                                                                           
 • Continuous Evolution: The video concludes by stating that search will continue to evolve using generative AI to 
   answer questions and present information in various formats, promising a more interactive and insightful search 
   experience.                                                                                                     

In essence, Google's generative AI aims to provide users with more than just a list of links. It aims to deliver a 
conversational, comprehensive, and visually engaging search experience that caters to complex questions and        
individual needs.

You can also use cached context to cache the video and keep asking questions for larger video: [Link](https://ai.google.dev/gemini-api/docs/caching?lang=python#generate-content)

## Embeddings

In [3]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/text-embedding-004'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding['embedding'][:5]) #printing only first 5

[-0.0021609126, -0.003164448, -0.060120765, -0.0071218405, 0.00087754615]


In [4]:
len(embedding['embedding'])

768